<a href="https://colab.research.google.com/github/shekharkhandelwal1983/DLforDataArchitects/blob/main/notebooks/chapter7/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate

def fcn8(input_shape, num_classes):
    inputs = Input(input_shape)

    # Block 1
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Block 2
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Block 3
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Block 4
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Block 5
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    # Fully Convolutionalization
    fc6 = Conv2D(4096, 7, activation='relu', padding='same')(pool5)
    fc6 = Dropout(0.5)(fc6)
    fc7 = Conv2D(4096, 1, activation='relu', padding='same')(fc6)
    fc7 = Dropout(0.5)(fc7)

    # Score Pooling 4
    score_pool4 = Conv2D(num_classes, 1, activation='relu', padding='same')(pool4)

    # Score Pooling 7
    score_fc7 = Conv2D(num_classes, 1, activation='relu', padding='same')(fc7)

    # Deconvolution 2x
    upsample_2x = Conv2DTranspose(num_classes, 4, strides=(2, 2), padding='same')(score_fc7)

    # Score Sum 2
    score_sum2 = concatenate([score_pool4, upsample_2x], axis=3)

    # Deconvolution 2x
    upsample_4x = Conv2DTranspose(num_classes, 4, strides=(2, 2), padding='same')(score_sum2)

    # Final Prediction
    upsample_final = Conv2DTranspose(num_classes, 16, strides=(8, 8), padding='same')(upsample_4x)
    outputs = keras.activations.softmax(upsample_final, axis=-1)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def generate_sample_data(num_samples, image_shape, num_classes):
    X = np.random.rand(num_samples, *image_shape)
    y = np.random.randint(0, num_classes, (num_samples, *image_shape[:-1], 1))
    return X, y

# Define input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 10

# Generate sample training data
X_train, y_train = generate_sample_data(100, input_shape, num_classes)

# Generate sample test data
X_test, y_test = generate_sample_data(20, input_shape, num_classes)

# Create FCN8 model
model = fcn8(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
7/7 [==============================] - 430s 59s/step - loss: 2.3026 - accuracy: 0.1001 - val_loss: 2.3026 - val_accuracy: 0.1002
Epoch 2/10
7/7 [==============================] - 405s 57s/step - loss: 2.3026 - accuracy: 0.1006 - val_loss: 2.3026 - val_accuracy: 0.1003
Epoch 3/10
7/7 [==============================] - 424s 60s/step - loss: 2.3026 - accuracy: 0.1009 - val_loss: 2.3026 - val_accuracy: 0.1003
Epoch 4/10
7/7 [==============================] - 411s 57s/step - loss: 2.3026 - accuracy: 0.1013 - val_loss: 2.3026 - val_accuracy: 0.1003
Epoch 5/10
7/7 [==============================] - 411s 58s/step - loss: 2.3025 - accuracy: 0.1019 - val_loss: 2.3027 - val_accuracy: 0.0998
Epoch 6/10
7/7 [==============================] - 401s 57s/step - loss: 2.3025 - accuracy: 0.1025 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 7/10
7/7 [==============================] - 413s 57s/step - loss: 2.3024 - accuracy: 0.1029 - val_loss: 2.3028 - val_accuracy: 0.1000
Epoch 8/10
7/7 [====